<a href="https://colab.research.google.com/github/anilpaudel/testing/blob/master/recommend_from_pickle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pickle

In [0]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from lightfm import LightFM
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix 
from scipy.sparse import coo_matrix 
import time
from datetime import datetime, timedelta
from sklearn.metrics import roc_auc_score
from lightfm.evaluation import auc_score
import pickle
import re

In [0]:

def similar_recommendation(model, interaction_matrix, user_id, user_dikt, 
                               item_dikt,threshold = 0,number_rec_items = 15):

    #Function to produce user recommendations

    n_users, n_items = interaction_matrix.shape
    user_x = user_dikt[user_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_items)))
    scores.index = interaction_matrix.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interaction_matrix.loc[user_id,:][interaction_matrix.loc[user_id,:] > threshold].index).sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    score_list = scores[0:number_rec_items]
 
    known_items = list(pd.Series(known_items).apply(lambda x: item_dikt[x]))
    scores = list(pd.Series(score_list).apply(lambda x: item_dikt[x]))
    scores1 = list(pd.Series(score_list))

#     jsonScores = json.dumps(scores)
#     print(jsonScores)
    
#     return json.dumps(scores)
    client=pymongo.MongoClient('mongodb://110.34.31.28:27017/')
    mydb=client['majorProject']
    mycol=mydb['bookDataset']
    x=mycol.find({"ISBN":{"$in":scores1}},
                {'_id':0,'ISBN':1,'Book-Title':1,'Book-Author':1,'Image-URL':1,'description':1   })
    y=[]
    for i in range(15):
      y.append(x[i])
    
    
    print(scores1)
    print(scores)
    # new dataframe for books
    book_newdf = pd.DataFrame({'bookTitle':scores})
   
    print("Items that were liked by the User:")
    counter = 1
    for i in known_items[:25]:
        print(str(counter) + '- ' + i)
        counter+=1

    print("\n Recommended Items:")
    counter = 1
    for i in scores:
        print(str(counter) + '- ' + i)
        counter+=1
    return book_newdf,y

In [0]:
with open('model.pickle', 'rb') as handle:
    model_pickle = pickle.load(handle)

In [0]:
with open('item_dikt.pickle', 'rb') as handle:
   item_dikt_pickle = pickle.load(handle)

In [0]:
with open('user_item_matrix.pickle', 'rb') as handle:
    user_item_matrix_pickle = pickle.load(handle)

In [0]:
with open('user_dikt.pickle', 'rb') as handle:
    user_dikt_pickle = pickle.load(handle)

In [10]:
print(item_dikt_pickle)

{'0374157065': 'Flu: The Story of the Great Influenza Pandemic of 1918 and the Search for the Virus That Caused It', '0399135782': "The Kitchen God's Wife", '0425176428': "What If?: The World's Foremost Military Historians Imagine What Might Have Been", '1575663937': 'More Cunning Than Man: A Social History of Rats and Man', '0440234743': 'The Testament', '0452264464': 'Beloved (Plume Contemporary Fiction)', '0609804618': "Our Dumb Century: The Onion Presents 100 Years of Headlines from America's Finest News Source", '1879384493': "If I'd Known Then What I Know Now: Why Not Learn from the Mistakes of Others? : You Can't Afford to Make Them All Yourself", '0689821166': 'Flood : Mississippi 1927', '0971880107': 'Wild Animus', '0345417623': 'Timeline', '0684823802': 'OUT OF THE SILENT PLANET', '0375759778': 'Prague : A Novel', '3442410665': 'Sturmzeit. Roman.', '0060168013': 'Pigs in Heaven', '038078243X': "Miss Zukas and the Raven's Dance", '067176537X': 'The Therapeutic Touch: How to Us

In [0]:
client=pymongo.MongoClient('mongodb://110.34.31.28:27017/')
mydb=client['majorProject']
mycol=mydb['bookDataset']

In [43]:
book_df,y = similar_recommendation(model_pickle, user_item_matrix_pickle,507 , user_dikt_pickle, item_dikt_pickle,threshold = 7)


['0312983271', '0743418174', '0312195516', '0385335482', '0671027360', '0316601950', '0312980140', '014028009X', '044021145X', '0804106304', '0449219364', '0312265859', '0140293248', '0380718340', '0446606812']
["Full House (Janet Evanovich's Full Series)", 'Good in Bed', 'The Red Tent (Bestselling Backlist)', 'Confessions of a Shopaholic (Summer Display Opportunity)', 'Angels &amp; Demons', "The Pilot's Wife : A Novel", 'Seven Up (A Stephanie Plum Novel)', "Bridget Jones's Diary", 'The Firm', 'The Joy Luck Club', 'G Is for Gumshoe (Kinsey Millhone Mysteries (Paperback))', 'Hard Eight : A Stephanie Plum Novel (A Stephanie Plum Novel)', "The Girls' Guide to Hunting and Fishing", 'Cruel &amp; Unusual (Kay Scarpetta Mysteries (Paperback))', 'Message in a Bottle']
Items that were liked by the User:
1- The Morning After
2- Silver on the Tree
3- The Grey King
4- Greenwitch
5- The Dark Is Rising
6- The Rosewood Casket
7- Colman
8- The Clinic
9- If Ever I Return, Pretty Peggy-O
10- In the Blea

In [44]:
print(y)

[{'ISBN': '0804106304', 'Book-Title': 'The Joy Luck Club', 'Book-Author': 'Amy Tan', 'Image-URL': 'http://images.amazon.com/images/P/0804106304.01.LZZZZZZZ.jpg', 'description': 'This widely acclaimed bestseller spans two countries and two generations, following a group of Chinese women who meet to play mah jong, invest money and tell the secret stories of their lives. They call their gathering the Joy Luck Club.'}, {'ISBN': '0671027360', 'Book-Title': 'Angels &amp; Demons', 'Book-Author': 'Dan Brown', 'Image-URL': 'http://images.amazon.com/images/P/0671027360.01.LZZZZZZZ.jpg', 'description': 'See alternate cover edition \x01\nWorld-renowned Harvard symbologist Robert Langdon is summoned to a Swiss research facility to analyze a cryptic symbol seared into the chest of a murdered physicist. What he discovers is unimaginable: a deadly vendetta against the Catholic Church by a centuries-old underground organization--- the Illuminati. Desperate to save the Vatican from a powerful time bomb,

In [0]:
0374157065

In [16]:
pip install pymongo

In [0]:
import pymongo

In [0]:
client=pymongo.MongoClient('mongodb://110.34.31.28:27017/')
mydb=client['majorProject']
mycol=mydb['bookDataset']
x=mycol.find({"ISBN":{"$in":['0312983271', '0743418174', '0312195516', '0385335482', '0671027360', '0316601950', '0312980140', '014028009X', '044021145X', '0804106304', '0449219364', '0312265859', '0140293248', '0380718340', '0446606812']}},
                {'_id':0,'ISBN':1,'Book-Title':1,'Book-Author':1,'Image-URL':1,'description':1   })

In [38]:
for i in range(15):
  print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [0]:
for i in range(len(x))

In [41]:
print(x[1])

{'ISBN': '0671027360', 'Book-Title': 'Angels &amp; Demons', 'Book-Author': 'Dan Brown', 'Image-URL': 'http://images.amazon.com/images/P/0671027360.01.LZZZZZZZ.jpg', 'description': 'See alternate cover edition \x01\nWorld-renowned Harvard symbologist Robert Langdon is summoned to a Swiss research facility to analyze a cryptic symbol seared into the chest of a murdered physicist. What he discovers is unimaginable: a deadly vendetta against the Catholic Church by a centuries-old underground organization--- the Illuminati. Desperate to save the Vatican from a powerful time bomb, Langdon joins forces in Rome with the beautiful and mysterious scientist Vittoria Vetra. Together they embark on a frantic hunt through sealed crypts, dangerous catacombs, deserted cathedrals, and the most secretive vault on the earth... the long-forgotten Illuminati lair.'}
